In [3]:
from qibo import Circuit, gates

In [4]:
def build_circuit(nqubits, nlayers):
    """Build qibo's aavqe example circuit."""

    circuit = Circuit(nqubits)
    for _ in range(nlayers):
        circuit.add(gates.RY(q, theta=0) for q in range(nqubits))
        circuit.add(gates.RZ(q, theta=0) for q in range(nqubits))
        circuit.add(gates.CZ(q, q + 1) for q in range(0, nqubits - 1, 2))
        circuit.add(gates.RY(q, theta=0) for q in range(nqubits))
        circuit.add(gates.RZ(q, theta=0) for q in range(nqubits))
        circuit.add(gates.CZ(q, q + 1) for q in range(1, nqubits - 2, 2))
        circuit.add(gates.CZ(0, nqubits - 1))
    circuit.add(gates.RY(q, theta=0) for q in range(nqubits))

    return circuit

In [7]:
nqubits = 10
nlayers = 15

c = build_circuit(nqubits, nlayers)

In [8]:
print(c.summary())

Circuit depth = 91
Total number of gates = 760
Number of qubits = 10
Most common gates:
ry: 310
rz: 300
cz: 150


#### Pure VQE training

In the following we consider results with equal accuracy in the ground state approximation.

We call $N_{2q}$ the total number of 2-qubit gates of a VQE.

**Worst case**: gradient descent on hardware. We need 2 expectation values for each parameter of the circuit to compute the derivative. Then you need to compute it for each epoch of training. The total number of circuit execution to evaluate the gradient is $K=2p$, with $p$ number of params.

Considering a training of $N_{ep}$ the total cost is $N_{ep}\cdot t_{1c} \cdot K$.

$$ N_{tot} = K \cdot N_{2q} \cdot N_{ep} = N_{vqe} \cdot N_{ep}$$ 

In the best case scenario the loss function requires only one expected value for each optimization iteration and then $K=1$.

#### Hybrid VQE-DBI training

The cost of one epoch of VQE training can be different if the size of the VQE will change. 

$$ N'_{tot} = K' \cdot N'_{2q} \cdot N'_{ep} + N_{2q, dbi}(n_{step}) = N'_{vqe} \cdot N'_{ep} + N_{2q, dbi}(n_{step})$$

#### Metric

We define the metric 

$$ \chi = \frac{N_{tot}}{N'_{tot}} = \frac{N_{vqe} \cdot N_{ep}}{N'_{vqe} \cdot N'_{ep} + N_{2q, dbi}(n_{step})}. $$ 

In which configuration it happens $\chi \geq 1$?

#### Evaluation procedure

1. The number of qubits is fixed by the target Hamiltonian. Ideally enough large to ensure presence of Barren Plateaus;
2. The number of DBI steps is fixed so that the number of 2-qubit gates doesn't explode; 
3. Fix $\varepsilon$ as target accuracy in the energy approximation;
    1. **Start pure VQE trainig:** we repeat the training, changing the number of layers and epochs. We select the models that can approximate $E_0$ with accuracy $\leq \varepsilon$, and we pick the epoch-layers couple that minimize $N_{tot}$. In this way $N_{tot}$ is fixed.
    3. **Start hybrid training:** run VQE trainings for increasing number of layers and epochs until the accuracy is $\leq \varepsilon$;
    4. evaluate $N'_{tot}$.
    5. Evaluate the $\chi$ matrix varying layers and epochs of the hybrid training.

With the matrix we could evaluate the regime where the hybrid training has similar performances of the pure training in terms of accuracy and number of gates used.